In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn import metrics

**DATA UPLOAD BY USING GOOGLE COLAB PACKAGE**

In [0]:
from google.colab import files 
uploaded = files.upload() 

Saving Dataset.xlsx to Dataset.xlsx


In [0]:
import io 
import pandas as pd
df = pd.read_excel('Dataset.xlsx')


In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.head(10)

,TXS_DATE,ACCOUNTID,ACCOUNTNAME,MERCHANTID,MERCHANTNAME,PAYMENTPRODUCTID,PAYMENTPRODUCTNAME,PAYMENTMETHODID,PAYMENTMETHODNAME,CARDSCHEME,CURRENCYCODE,CURRENCY_TYPE,COUNTRYCODE,IINCOUNTRYCODE,INDUSTRY,RECURRING_INDICATOR,TRX_3D_TYPE,TRX_3D_RESULT,NUMBER_TXS_CREATED,NUMBER_TXS_AUTHORIZED,NUMBER_TXS_PAID,NUMBER_REF_CREATED,NUMBER_TXS_REVERSED
0,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,1,Visa Online,10,Credit cards,VISA,USD,Foreign currency,BR,BR,Travel,N,No 3D,Not applicable,4,4,13,0,0
1,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,119,Mastercard Debit,10,Credit cards,ECMC,USD,Foreign currency,SE,SE,Travel,N,No 3D,Not applicable,1,1,1,0,0
2,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,114,Visa Debit,10,Credit cards,VISA,EUR,Foreign currency,MX,MX,Travel,N,No 3D,Not applicable,0,0,1,0,0
3,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,114,Visa Debit,10,Credit cards,VISA,GBP,Foreign currency,CH,GB,Travel,N,No 3D,Not applicable,0,0,1,0,0
4,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,1,Visa Online,10,Credit cards,VISA,GBP,Local currency,GB,GB,Travel,Y,3D done by Ingenico,Failed,6,6,4,0,0
5,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,2,American Express Online,10,Credit cards,AMEX,CAD,Local currency,CA,CA,Travel,Y,No 3D,Not applicable,0,0,3,0,0
6,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,2,American Express Online,10,Credit cards,AMEX,CAD,Local currency,CA,SG,Travel,Y,No 3D,Not applicable,0,0,1,0,0
7,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,1,Visa Online,10,Credit cards,VISA,GBP,Local currency,GB,GB,Travel,Y,3D done by Ingenico,Not applicable,1,1,1,0,0
8,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,3,MasterCard Online,10,Credit cards,ECMC,USD,Local currency,US,US,Travel,N,No 3D,Not applicable,29,26,21,1,0
9,2013-11-28,3,ACME Retailer,-4,ACME Entreprise DEV,1,Visa Online,10,Credit cards,VISA,GBP,Foreign currency,FR,FR,Travel,N,No 3D,Not applicable,0,0,1,0,0


**DATA EXPLORATION**

> First check numeric data statistics quickly then explore the interesting categorical features by calculating distinct values for each.



In [0]:
df.describe()

,ACCOUNTID,MERCHANTID,PAYMENTPRODUCTID,PAYMENTMETHODID,NUMBER_TXS_CREATED,NUMBER_TXS_AUTHORIZED,NUMBER_TXS_PAID,NUMBER_REF_CREATED,NUMBER_TXS_REVERSED
count,113157.000000,113157.0,113157.000000,113157.000000,113157.000000,113157.000000,113157.000000,113157.000000,113157.000000
mean,3.099075,-4.0,89.591090,14.049948,4.527895,3.682406,6.331981,0.251836,0.122723
std,1.225251,0.0,242.491225,13.386440,17.022027,15.039029,25.283306,1.611538,1.535897
min,1.000000,-4.0,1.000000,10.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,-4.0,1.000000,10.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,3.000000,-4.0,3.000000,10.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,-4.0,3.000000,10.000000,3.000000,2.000000,4.000000,0.000000,0.000000
max,5.000000,-4.0,1506.000000,80.000000,866.000000,789.000000,1508.000000,71.000000,67.000000


In [0]:
field_counts = {}
for field in df.columns:
    try:
        field_counts[field] = df[field].nunique()
    except:
        print('count failed!', field)

In [0]:
interesting_field_counts = {k:v for k, v in field_counts.items() if v>1}

interesting_field_counts

{'ACCOUNTID': 5,
 'ACCOUNTNAME': 5,
 'CARDSCHEME': 7,
 'COUNTRYCODE': 185,
 'CURRENCYCODE': 25,
 'CURRENCY_TYPE': 2,
 'IINCOUNTRYCODE': 165,
 'NUMBER_REF_CREATED': 55,
 'NUMBER_TXS_AUTHORIZED': 265,
 'NUMBER_TXS_CREATED': 285,
 'NUMBER_TXS_PAID': 414,
 'NUMBER_TXS_REVERSED': 50,
 'PAYMENTMETHODID': 7,
 'PAYMENTMETHODNAME': 7,
 'PAYMENTPRODUCTID': 27,
 'PAYMENTPRODUCTNAME': 27,
 'RECURRING_INDICATOR': 2,
 'TRX_3D_RESULT': 5,
 'TRX_3D_TYPE': 4,
 'TXS_DATE': 310}

The above fields might be possible features to be used in modeling since it makes sense from the end of variabilities.

The following output shows also the summary of the distribution (frequency) for each value for each feature.

In [0]:
for field in sorted(interesting_field_counts.keys()):
    
    try: 
        df_count = (
            df
            .groupby([field])[field]
            .count()
            .reset_index(name='count')
            .sort_values(['count'], ascending=[False]))

        print("Showing distribution for: ", field)
        print('Number of unique values: ', df_count.count())
        print(df_count.head(5))
        
    except:
        print("Fail to show: ", field)

Showing distribution for:  ACCOUNTID
Number of unique values:  ACCOUNTID    5
count        5
dtype: int64
   ACCOUNTID  count
2          3  50091
4          5  22455
1          2  16878
0          1  13518
3          4  10215
Showing distribution for:  ACCOUNTNAME
Number of unique values:  ACCOUNTNAME    5
count          5
dtype: int64
     ACCOUNTNAME  count
3  ACME Retailer  50091
2   ACME Publish  22455
4    ACME Travel  16878
1      ACME Game  13518
0   ACME Finance  10215
Showing distribution for:  CARDSCHEME
Number of unique values:  CARDSCHEME    7
count         7
dtype: int64
  CARDSCHEME  count
6       VISA  55206
3       ECMC  37048
0       AMEX   8794
4        JCB    763
1         CB    460
Showing distribution for:  COUNTRYCODE
Number of unique values:  COUNTRYCODE    185
count          185
dtype: int64
    COUNTRYCODE  count
85           IT  12427
127          NL   9677
170          US   9118
57           GB   8181
42           DE   4874
Showing distribution for:  CURRENCY

**FEATURE SELECTION**

Since we know that from the data definition and also from the sample data the following features have the same meaning in data (one of them refers to id other one refers to the name) we can just consider one of them in model colums categorization.  

1.   ACCOUNTID = ACCOUNTNAME
2.   PAYMENTMETHODID = PAYMENTMETHODNAME
3.   PAYMENTPRODUCTID = PAYMENTPRODUCTNAME





In [0]:
categorical_columns = [

###potential model columns
    'ACCOUNTID', #or ACCOUNTNAME
    'CARDSCHEME',
    'COUNTRYCODE',
    'CURRENCYCODE',
    'CURRENCY_TYPE',
    'IINCOUNTRYCODE',
    'PAYMENTMETHODID', #or PAYMENTMETHODNAME
    'PAYMENTPRODUCTID', #or PAYMENTPRODUCTNAME
    'RECURRING_INDICATOR',
    'TRX_3D_RESULT',
    'TRX_3D_TYPE'
]


numeric_columns = [
    'NUMBER_REF_CREATED',
    'NUMBER_TXS_AUTHORIZED',
    'NUMBER_TXS_CREATED',
    'NUMBER_TXS_PAID',
    'NUMBER_TXS_REVERSED'
]
   


**DATA VISUALIZATION**

It is a good idea to see the distributions of the reversed transaction counts across categorical features. For this case I prefered to discover just reversed rows in data to see the difference more clear. 

In [0]:
condition = df['NUMBER_TXS_REVERSED'] > 0

df_2 = df[condition]

In [0]:
import plotly.express as px

It can be said by looking at the following plot: ACME Game account name has a wide distribution and high median from the end of reversed transactions. When we check the same graph for number of paid transactions it can be said this account is still making difference relative to other ones but much more less effect than happened like in reversed transactions. I can conclude that Account name is important for the reversed transactions just looking at these graphs. 

ACME Retailer is following up ACME Game and than the others. 

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="ACCOUNTNAME"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="ACCOUNTNAME"
        , title="Distribution of Number of Txs paid"
        , template='presentation'
        )

It can be said by looking at the following plot: The payment transactions that are recurring seem more likely to be reversed later.

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="RECURRING_INDICATOR"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="RECURRING_INDICATOR"
        , title="Distribution of Number of Txs Paid"
        , template='presentation'
        )

The following plot shows us the distribution of Account name and Recurring flag effect at the same time. 

ACME Game account name and payment transactions with recurring indicator yes seem more likely to high reverse transactions. 

On the other hand ACME Retailer account name and payment transactions with non-recurring type seem more likely to high reverse transactions.

When looking at the second graph with number of paid transactions we can say that the number of reversed transactions seem also relational to the number of paid transactions.

In [0]:
fig = px.histogram(df_2, x="NUMBER_TXS_REVERSED",  color='RECURRING_INDICATOR', facet_row="ACCOUNTNAME")
fig.update_yaxes(matches=None)
fig.update_layout(
    title={
        'text': "Histogram of Number of Txs Reversed Across Account by Recurring Ind",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

for a in fig.layout.annotations:
    a.text = a.text.split("=")[1]
fig.show()


In [0]:
fig = px.histogram(df_2, x="NUMBER_TXS_PAID",  color='RECURRING_INDICATOR', facet_row="ACCOUNTNAME")
fig.update_yaxes(matches=None)
fig.update_layout(
    title={
        'text': "Histogram of Number of Txs Paid Across Account by Recurring Ind",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

for a in fig.layout.annotations:
    a.text = a.text.split("=")[1]
fig.show()


It can be said by looking at the following plot: Some payment products are more likely to high volume reversed transactions like DDDE Recurring,DDNL Doorlopende machtiging Nederland, DDES Recurring..




In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="PAYMENTPRODUCTNAME"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="PAYMENTPRODUCTNAME"
        , title="Distribution of Number of Txs paid"
        , template='presentation'
        )

It can be said by looking at the following plot: Payment method with Direct debit seems more likely to high volume reversed transactions.

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="PAYMENTMETHODNAME"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="PAYMENTMETHODNAME"
        , title="Distribution of Number of Txs paid"
        , template='presentation'
        )

It can be said by looking at the following plot: There does not seem huge difference across different cardschemes but it seems a bit high probability for the DISC one and high values for the VISA.

In [0]:
px.box(data_frame=df_2[~df_2.CARDSCHEME.isna()]
        , y="NUMBER_TXS_REVERSED"
        , color="CARDSCHEME"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

It can be said by looking at the following 2 plots: 
For some currency codes like EUR, GBP, USD the reversed transactions are much more higher than others. But it might be also related to the high paid transactions. (same conclusion for Currency type : local currency)

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="CURRENCYCODE"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="CURRENCYCODE"
        , title="Distribution of Number of Txs paid"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="CURRENCY_TYPE"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="CURRENCY_TYPE"
        , title="Distribution of Number of Txs paid"
        , template='presentation'
        )

In [0]:
It can be said by looking at the following 2 plots: 
For some country codes like DE, NL, ES the reversed transactions are much more higher than others. 
But it might be also related to the high paid transactions especially for NL and DE. ES seems the most relevant one from the end of paid transactions for the reversal likelihood.

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="COUNTRYCODE"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="COUNTRYCODE"
        , title="Distribution of Number of Txs paid"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="IINCOUNTRYCODE"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_PAID"
        , color="IINCOUNTRYCODE"
        , title="Distribution of Number of Txs paid"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="TRX_3D_TYPE"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

In [0]:
px.box(data_frame=df_2
        , y="NUMBER_TXS_REVERSED"
        , color="TRX_3D_RESULT"
        , title="Distribution of Number of Txs reversed"
        , template='presentation'
        )

**FEATURE ENGINEERING**

In [0]:
df['year_month'] = df['TXS_DATE'].dt.year.astype(str) + df['TXS_DATE'].dt.month.astype(str)

In [0]:
group_columns = [

    'ACCOUNTID', #or ACCOUNTNAME
    'CARDSCHEME',
    'COUNTRYCODE',
    'CURRENCYCODE',
    'CURRENCY_TYPE',
    'IINCOUNTRYCODE',
    'PAYMENTMETHODID', #or PAYMENTMETHODNAME
    'PAYMENTPRODUCTID', #or PAYMENTPRODUCTNAME
    'RECURRING_INDICATOR',
    'TRX_3D_RESULT',
    'TRX_3D_TYPE',
    "year_month"
] 

df_count = (
    df
    .groupby(group_columns, as_index= False)
    .agg({"NUMBER_TXS_CREATED": "sum",
          "NUMBER_TXS_AUTHORIZED": "sum",
          "NUMBER_REF_CREATED": "sum",
          "NUMBER_TXS_PAID": "sum",
          "NUMBER_TXS_REVERSED": "sum",
          "TXS_DATE": pd.Series.nunique  
          }))


df_count["PR_TXS_AUTHORIZED"] = df_count['NUMBER_TXS_AUTHORIZED'] / df_count['NUMBER_TXS_CREATED']
df_count["PR_TXS_AUTHORIZED"].fillna(1, inplace=True)
df_count["NUMBER_TXS_PAID_PER_DAY"] = df_count['NUMBER_TXS_PAID'] / df_count['TXS_DATE']
df_count["NUMBER_TXS_REVERSED_PER_DAY"] = df_count['NUMBER_TXS_REVERSED'] / df_count['TXS_DATE']
df_count["NUMBER_REF_CREATED_PER_DAY"] = df_count['NUMBER_REF_CREATED'] / df_count['TXS_DATE']

print('Number of rows', len(df_count))
df_count.head(5)

Number of rows 25346


,ACCOUNTID,CARDSCHEME,COUNTRYCODE,CURRENCYCODE,CURRENCY_TYPE,IINCOUNTRYCODE,PAYMENTMETHODID,PAYMENTPRODUCTID,RECURRING_INDICATOR,TRX_3D_RESULT,TRX_3D_TYPE,year_month,NUMBER_TXS_CREATED,NUMBER_TXS_AUTHORIZED,NUMBER_REF_CREATED,NUMBER_TXS_PAID,NUMBER_TXS_REVERSED,TXS_DATE,PR_TXS_AUTHORIZED,NUMBER_TXS_PAID_PER_DAY,NUMBER_TXS_REVERSED_PER_DAY,NUMBER_REF_CREATED_PER_DAY
0,1,AMEX,AR,USD,Foreign currency,MX,10,2,N,Not applicable,No 3D,20131,0,0,0,1,0,1,1.0,1.0,0.0,0.0
1,1,AMEX,AT,EUR,Local currency,AT,10,2,Y,Not applicable,No 3D,20131,0,0,0,2,0,1,1.0,2.0,0.0,0.0
2,1,AMEX,AT,EUR,Local currency,AT,10,2,Y,Not applicable,No 3D,201311,0,0,0,1,0,1,1.0,1.0,0.0,0.0
3,1,AMEX,AT,EUR,Local currency,AT,10,2,Y,Not applicable,No 3D,201312,0,0,0,1,0,1,1.0,1.0,0.0,0.0
4,1,AMEX,AT,EUR,Local currency,AT,10,2,Y,Not applicable,No 3D,20132,0,0,0,2,0,2,1.0,1.0,0.0,0.0


In [0]:
def counting(df, col_name, verbose=0, percentile=False):
    '''
    Counting the number of unique values of a column.
    col_name: str, column name to be counted.
    verbose: 0 or 1, print out some information if 1.
    percentile: True or False, if True, the counts is normalized into ratio (between 0 and 1).
    '''
    col_name_count = col_name+"_count"
        
    df_count = (df[col_name]
                .groupby(df[col_name]).count()
                .reset_index(name=col_name_count).sort_values([col_name_count], ascending=False))
    
    if 1==verbose:
        print('Number of unique values: ', len(df_count))
       
    if percentile:
        df_count[col_name_count] = df_count[col_name_count]/len(df)
        
    return df_count

    
def indexing(df, col_name, offset=0):
    
    '''
    Indexing categorical columns
    col_name: str, the column to be indexed
    offset: int, the starting value of indexing
    '''
    
    df_count_index = counting(df, col_name)  
    df_count_index[col_name+"_index"] = np.array(range(len(df_count_index))) + offset
    return pd.merge(left=df, right=df_count_index, on=col_name)

In [0]:
# indexing categorical columns
for col_name in categorical_columns:
    df_count = indexing(df_count, col_name=col_name, offset=1)

In [0]:
#once
for col_name in categorical_columns:
    
    try:
        print(counting(df_count, col_name, verbose=1).head(5))
    except:
        print('Failed to show counts for:', col_name)
    
    print('\n')

Number of unique values:  5
   ACCOUNTID  ACCOUNTID_count
2          3            11458
4          5             4882
1          2             3864
0          1             2793
3          4             2349


Number of unique values:  7
  CARDSCHEME  CARDSCHEME_count
6       VISA             13319
3       ECMC              8759
0       AMEX              2976
4        JCB               179
5       SWIT                75


Number of unique values:  185
    COUNTRYCODE  COUNTRYCODE_count
85           IT               3075
170          US               2186
127          NL               1799
57           GB               1699
42           DE                868


Number of unique values:  18
   CURRENCYCODE  CURRENCYCODE_count
17          USD               11107
8           EUR                9931
9           GBP                2278
3           CAD                 416
1           AUD                 326


Number of unique values:  2
      CURRENCY_TYPE  CURRENCY_TYPE_count
1    Local curre

In [0]:
categorical_columns = [

###potential model columns
    'ACCOUNTID_index', #or ACCOUNTNAME
    'CARDSCHEME_index',
    'COUNTRYCODE_index',
    'CURRENCYCODE_index',
    'CURRENCY_TYPE_index',
    'IINCOUNTRYCODE_index',
    'PAYMENTMETHODID_index', #or PAYMENTMETHODNAME
    'PAYMENTPRODUCTID_index', #or PAYMENTPRODUCTNAME
    'RECURRING_INDICATOR_index',
    'TRX_3D_RESULT_index',
    'TRX_3D_TYPE_index'
]


numeric_columns = [
    'NUMBER_REF_CREATED_PER_DAY',
    'PR_TXS_AUTHORIZED',
    'NUMBER_TXS_PAID_PER_DAY'
]
   

target_column = ['NUMBER_TXS_REVERSED_PER_DAY']

In [0]:
df_count[target_column+numeric_columns].describe()

,NUMBER_TXS_REVERSED_PER_DAY,NUMBER_REF_CREATED_PER_DAY,PR_TXS_AUTHORIZED,NUMBER_TXS_PAID_PER_DAY
count,25346.000000,25346.000000,25346.000000,25346.000000
mean,0.003940,0.060639,0.956862,2.290148
std,0.060226,0.593066,0.158600,9.215235
min,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,1.000000,1.000000
50%,0.000000,0.000000,1.000000,1.000000
75%,0.000000,0.000000,1.000000,1.500000
max,3.000000,27.739130,1.000000,587.727273


In [0]:
df_count.sort_values("NUMBER_TXS_REVERSED_PER_DAY", ascending=False)

,ACCOUNTID,CARDSCHEME,COUNTRYCODE,CURRENCYCODE,CURRENCY_TYPE,IINCOUNTRYCODE,PAYMENTMETHODID,PAYMENTPRODUCTID,RECURRING_INDICATOR,TRX_3D_RESULT,TRX_3D_TYPE,year_month,NUMBER_TXS_CREATED,NUMBER_TXS_AUTHORIZED,NUMBER_REF_CREATED,NUMBER_TXS_PAID,NUMBER_TXS_REVERSED,TXS_DATE,PR_TXS_AUTHORIZED,NUMBER_TXS_PAID_PER_DAY,NUMBER_TXS_REVERSED_PER_DAY,NUMBER_REF_CREATED_PER_DAY,ACCOUNTID_count,ACCOUNTID_index,CARDSCHEME_count,CARDSCHEME_index,COUNTRYCODE_count,COUNTRYCODE_index,CURRENCYCODE_count,CURRENCYCODE_index,CURRENCY_TYPE_count,CURRENCY_TYPE_index,IINCOUNTRYCODE_count,IINCOUNTRYCODE_index,PAYMENTMETHODID_count,PAYMENTMETHODID_index,PAYMENTPRODUCTID_count,PAYMENTPRODUCTID_index,RECURRING_INDICATOR_count,RECURRING_INDICATOR_index,TRX_3D_RESULT_count,TRX_3D_RESULT_index,TRX_3D_TYPE_count,TRX_3D_TYPE_index
25195,1,VISA,US,USD,Local currency,US,10,1,N,Attempted partially passed,3D done by Ingenico,201310,0,0,0,3,6,2,1.000000,1.500000,3.000000,0.000000,2793,4,13319,1,2186,2,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,211,4,4297,2
25204,1,VISA,US,USD,Local currency,US,10,1,N,Attempted partially passed,3D done by Ingenico,20138,2,2,0,22,32,11,1.000000,2.000000,2.909091,0.000000,2793,4,13319,1,2186,2,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,211,4,4297,2
8480,3,VISA,US,USD,Local currency,US,10,1,N,Not applicable,No 3D,201311,1316,1163,143,1920,45,18,0.883739,106.666667,2.500000,7.944444,11458,1,13319,1,2186,2,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,21292,1,21034,1
8481,3,VISA,US,USD,Local currency,US,10,1,N,Not applicable,No 3D,201312,887,776,110,1461,37,15,0.874859,97.400000,2.466667,7.333333,11458,1,13319,1,2186,2,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,21292,1,21034,1
8479,3,VISA,US,USD,Local currency,US,10,1,N,Not applicable,No 3D,201310,1751,1548,196,2227,46,21,0.884066,106.047619,2.190476,9.333333,11458,1,13319,1,2186,2,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,21292,1,21034,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8538,4,VISA,GU,USD,Local currency,US,10,1,N,Not applicable,No 3D,20139,0,0,0,1,0,1,1.000000,1.000000,0.000000,0.000000,2349,5,13319,1,19,111,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,21292,1,21034,1
8537,4,VISA,GU,USD,Local currency,US,10,1,N,Not applicable,No 3D,20138,0,0,0,1,0,1,1.000000,1.000000,0.000000,0.000000,2349,5,13319,1,19,111,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,21292,1,21034,1
8536,4,VISA,GU,USD,Local currency,US,10,1,N,Not applicable,No 3D,20136,0,0,0,1,0,1,1.000000,1.000000,0.000000,0.000000,2349,5,13319,1,19,111,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,21292,1,21034,1
8535,4,VISA,GU,USD,Local currency,US,10,1,N,Not applicable,No 3D,201310,0,0,0,1,0,1,1.000000,1.000000,0.000000,0.000000,2349,5,13319,1,19,111,11107,1,13328,1,1808,2,25346,1,11000,1,20423,1,21292,1,21034,1


In [0]:

# For those indexed columns, if they have index>max_index, we set index=max_index
# For now, we set max_index to be the number of Make_Model combinations.
indexed_columns = [col_name for col_name in df_count.columns if col_name.endswith('_index')] 

max_index = df_count['COUNTRYCODE_index'].max()

df_count[indexed_columns].clip(upper=max_index, inplace=True)

print(df_count[indexed_columns].max())

df_count.head(3)


ACCOUNTID_index                5
CARDSCHEME_index               7
COUNTRYCODE_index            185
CURRENCYCODE_index            18
CURRENCY_TYPE_index            2
IINCOUNTRYCODE_index         163
PAYMENTMETHODID_index          1
PAYMENTPRODUCTID_index        11
RECURRING_INDICATOR_index      2
TRX_3D_RESULT_index            5
TRX_3D_TYPE_index              3
dtype: int64


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:7540: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ACCOUNTID,CARDSCHEME,COUNTRYCODE,CURRENCYCODE,CURRENCY_TYPE,IINCOUNTRYCODE,PAYMENTMETHODID,PAYMENTPRODUCTID,RECURRING_INDICATOR,TRX_3D_RESULT,TRX_3D_TYPE,year_month,NUMBER_TXS_CREATED,NUMBER_TXS_AUTHORIZED,NUMBER_REF_CREATED,NUMBER_TXS_PAID,NUMBER_TXS_REVERSED,TXS_DATE,PR_TXS_AUTHORIZED,NUMBER_TXS_PAID_PER_DAY,NUMBER_TXS_REVERSED_PER_DAY,NUMBER_REF_CREATED_PER_DAY,ACCOUNTID_count,ACCOUNTID_index,CARDSCHEME_count,CARDSCHEME_index,COUNTRYCODE_count,COUNTRYCODE_index,CURRENCYCODE_count,CURRENCYCODE_index,CURRENCY_TYPE_count,CURRENCY_TYPE_index,IINCOUNTRYCODE_count,IINCOUNTRYCODE_index,PAYMENTMETHODID_count,PAYMENTMETHODID_index,PAYMENTPRODUCTID_count,PAYMENTPRODUCTID_index,RECURRING_INDICATOR_count,RECURRING_INDICATOR_index,TRX_3D_RESULT_count,TRX_3D_RESULT_index,TRX_3D_TYPE_count,TRX_3D_TYPE_index
0,1,AMEX,AR,USD,Foreign currency,MX,10,2,N,Not applicable,No 3D,20131,0,0,0,1,0,1,1.0,1.0,0.0,0.0,2793,4,2976,3,203,29,11107,1,12018,2,426,15,25346,1,2976,3,20423,1,21292,1,21034,1
1,3,AMEX,AR,USD,Foreign currency,MX,10,2,N,Not applicable,No 3D,201310,0,0,0,1,0,1,1.0,1.0,0.0,0.0,11458,1,2976,3,203,29,11107,1,12018,2,426,15,25346,1,2976,3,20423,1,21292,1,21034,1
2,3,AMEX,AR,USD,Foreign currency,MX,10,2,N,Not applicable,No 3D,201311,0,0,0,2,0,2,1.0,1.0,0.0,0.0,11458,1,2976,3,203,29,11107,1,12018,2,426,15,25346,1,2976,3,20423,1,21292,1,21034,1


**MODEL TRAINING**

***RANDOM FOREST***

In [0]:

def evaluate_regressor(X, y, reg, is_log_scale=False):
    
    y_pred = reg.predict(X)
    
    if is_log_scale:
        y_pred = np.exp(y_pred)
        
    print('RMSE: %.3f' % (np.sqrt(metrics.mean_squared_error(y_true=y, y_pred=y_pred))))
    print('MAE: %.3f' % (metrics.mean_absolute_error(y_true=y, y_pred=y_pred)))
    print('R2 score: %.3f' % (metrics.r2_score(y_true=y, y_pred=y_pred)))

    return y_pred

def show_feature_importance(col_names, feature_importances):
    
    data = list(zip(list(col_names), list(feature_importances)))

    df = pd.DataFrame(
        data = data,
        columns=['feature', 'importance']).sort_values('importance', ascending=False)
    
    df.head(20)
    
    return df

In [0]:
# define input features and label
columns_to_model = categorical_columns + numeric_columns + target_column 
    
print(len(df_count))

df_count = df_count.dropna(subset=columns_to_model)

print(len(df_count))

    
df_selected_columns = (
    df_count[columns_to_model])

df_X = df_selected_columns.drop(target_column, axis=1)

X_np = np.array(df_X)

y_true_np = np.array(df_selected_columns[target_column])

print(X_np.shape)
print(y_true_np.shape)
df_X.head(3)

25346
25346
(25346, 14)
(25346, 1)


,ACCOUNTID_index,CARDSCHEME_index,COUNTRYCODE_index,CURRENCYCODE_index,CURRENCY_TYPE_index,IINCOUNTRYCODE_index,PAYMENTMETHODID_index,PAYMENTPRODUCTID_index,RECURRING_INDICATOR_index,TRX_3D_RESULT_index,TRX_3D_TYPE_index,NUMBER_REF_CREATED_PER_DAY,PR_TXS_AUTHORIZED,NUMBER_TXS_PAID_PER_DAY
0,4,3,29,1,2,15,1,3,1,1,1,0.0,1.0,1.0
1,1,3,29,1,2,15,1,3,1,1,1,0.0,1.0,1.0
2,1,3,29,1,2,15,1,3,1,1,1,0.0,1.0,1.0


In [0]:
# cross-validation

is_log_scale = False
is_model_trained = False

kfold = KFold(n_splits=2, shuffle=True, random_state=1)

df_selected_columns.loc[:, 'y_pred'] = 0

# define model
reg_rf_cv = RandomForestRegressor(
    n_estimators=20, 
    max_depth=8, 
    min_samples_leaf=4,
    random_state=0,
    n_jobs = 20,
    max_features='auto',
    criterion='mse' # 'mae' is two slow to finish
)

for train_idx, test_idx in kfold.split(X_np, y_true_np):
    
    # train model
    if is_model_trained == False:
        reg_rf_cv.fit(X_np[train_idx], y_true_np[train_idx])
        is_model_trained = True
    
    # evaluate model
    y_pred = evaluate_regressor(
        X_np[test_idx], 
        y_true_np[test_idx], 
        reg_rf_cv, 
        is_log_scale=is_log_scale
    )
    
    df_selected_columns['y_pred'].iloc[test_idx] = y_pred

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:376: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RMSE: 0.045
MAE: 0.005
R2 score: 0.437
RMSE: 0.042
MAE: 0.004
R2 score: 0.511


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is t

**FEATURE IMPORTANCE**

In [0]:
show_feature_importance(
    col_names=df_X.columns, 
    feature_importances=reg_rf_cv.feature_importances_).round({'importance':3})

,feature,importance
13,NUMBER_TXS_PAID_PER_DAY,0.448
2,COUNTRYCODE_index,0.182
0,ACCOUNTID_index,0.137
5,IINCOUNTRYCODE_index,0.109
9,TRX_3D_RESULT_index,0.039
12,PR_TXS_AUTHORIZED,0.029
3,CURRENCYCODE_index,0.027
11,NUMBER_REF_CREATED_PER_DAY,0.014
1,CARDSCHEME_index,0.005
7,PAYMENTPRODUCTID_index,0.005


**XGBOOST**

In [0]:
categorical_columns = [

###potential model columns
    'ACCOUNTID', #or ACCOUNTNAME
    'CARDSCHEME',
    'COUNTRYCODE',
    'CURRENCYCODE',
    'CURRENCY_TYPE',
    'IINCOUNTRYCODE',
    'PAYMENTMETHODID', #or PAYMENTMETHODNAME
    'PAYMENTPRODUCTID', #or PAYMENTPRODUCTNAME
    'RECURRING_INDICATOR',
    'TRX_3D_RESULT',
    'TRX_3D_TYPE'
]


numeric_columns = [
    'NUMBER_REF_CREATED_PER_DAY',
    'PR_TXS_AUTHORIZED',
    'NUMBER_TXS_PAID_PER_DAY'
]
   

target_column = ['NUMBER_TXS_REVERSED_PER_DAY']

In [0]:
# define input features and label
columns_to_model = categorical_columns + numeric_columns + target_column 
    
print(len(df_count))

df_count = df_count.dropna(subset=columns_to_model)

print(len(df_count))

    
df_selected_columns = (
    df_count[columns_to_model])



25346
25346
(25346, 14)
(25346, 1)


,ACCOUNTID,CARDSCHEME,COUNTRYCODE,CURRENCYCODE,CURRENCY_TYPE,IINCOUNTRYCODE,PAYMENTMETHODID,PAYMENTPRODUCTID,RECURRING_INDICATOR,TRX_3D_RESULT,TRX_3D_TYPE,NUMBER_REF_CREATED_PER_DAY,PR_TXS_AUTHORIZED,NUMBER_TXS_PAID_PER_DAY
0,1,AMEX,AR,USD,Foreign currency,MX,10,2,N,Not applicable,No 3D,0.0,1.0,1.0
1,3,AMEX,AR,USD,Foreign currency,MX,10,2,N,Not applicable,No 3D,0.0,1.0,1.0
2,3,AMEX,AR,USD,Foreign currency,MX,10,2,N,Not applicable,No 3D,0.0,1.0,1.0


In [0]:
# model xgboost

import pandas as pd
import xgboost as xgboost
from xgboost import XGBRegressor
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

  
train_dataset = df_selected_columns
train_dataset[categorical_columns] = train_dataset[categorical_columns].astype('category')


train_dataset = pd.get_dummies(train_dataset, prefix_sep="_",columns=categorical_columns)





trlabel = train_dataset[target_column]
train = train_dataset.drop(target_column, axis=1)

X_train, X_test, y_train, y_test = train_test_split(train, trlabel, test_size=0.2, random_state=7)


model = xgboost.XGBRegressor(colsample_bytree=0.4603,
         gamma= 0.0468,                 
         learning_rate=0.04,
         max_depth=4,
         min_child_weight=1.7817,
         n_estimators=200,                                                                    
         reg_alpha=0.75,
         reg_lambda=0.45,
         subsample=0.5213,
         seed=42,
         objective = 'reg:linear',
         verbose_eval = False, 
         eval_metric = 'mae', 
         nfold = 5, 
         nthread = 8                  
         ) 


model.fit(X_train,y_train)
preds = model.predict(X_train)
X_train['predValue'] = preds
X_train['actualValue'] = y_train
preds_test = model.predict(X_test)
X_test['predValue'] = preds_test
X_test['actualValue'] = y_test


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[00:58:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [0]:
print('Train evaluation metrics:')
print('RMSE: %.3f' % (np.sqrt(metrics.mean_squared_error(y_true=y_train, y_pred=preds))))
print('MAE: %.3f' % (metrics.mean_absolute_error(y_true=y_train, y_pred=preds)))
print('R2 score: %.3f' % (metrics.r2_score(y_true=y_train, y_pred=preds)))

print('Test evaluation metrics:')
print('RMSE: %.3f' % (np.sqrt(metrics.mean_squared_error(y_true=y_test, y_pred=preds_test))))
print('MAE: %.3f' % (metrics.mean_absolute_error(y_true=y_test, y_pred=preds_test)))
print('R2 score: %.3f' % (metrics.r2_score(y_true=y_test, y_pred=preds_test)))


Train evaluation metrics:
RMSE: 0.033
MAE: 0.004
R2 score: 0.651
Test evaluation metrics:
RMSE: 0.063
MAE: 0.005
R2 score: 0.284


In [0]:
show_feature_importance(
    col_names=train.columns, 
    feature_importances=model.feature_importances_).round({'importance':4})

,feature,importance
6,ACCOUNTID_4,0.1846
274,IINCOUNTRYCODE_GB,0.0700
12,CARDSCHEME_JCB,0.0667
14,CARDSCHEME_VISA,0.0596
217,CURRENCYCODE_USD,0.0499
400,TRX_3D_RESULT_Not applicable,0.0495
185,COUNTRYCODE_US,0.0430
5,ACCOUNTID_3,0.0382
404,TRX_3D_TYPE_No 3D,0.0370
4,ACCOUNTID_2,0.0354


**DECISION**
Although XGBoost train model evaluation metrics seem better than Random forest model results train-test error difference is high and the model doesn't seem stable and robust. 

Even R2 metric is lower in Random Forest output I would prefer this model to improve by doing some additional work.